In [ ]:
import requests
import dateutil
from datetime import datetime, timedelta
from IPython.core.display import display, HTML

In [ ]:
def get_data(endpoint, page, sort):
    r = requests.get('https://%s/api/1/%s?page_size=%s&sort=%s&page=%s' % (DOMAIN, endpoint, PAGE_SIZE, sort, page))
    return r.json()['data']

In [ ]:
def get_last_items(endpoint, date_key='created_at', sort_key='-created'):

    got_everything = False
    results = []
    page = 1

    while not got_everything:
        data = get_data(endpoint, page, sort_key)
        for d in data:
            created = dateutil.parser.parse(d[date_key])
            got_everything = (created < start_date)
            if not got_everything:
                results.append(d)
            else:
                break
        if got_everything:
            break
        else:
            page += 1

    return results

In [ ]:
def show_html(html):
    display(HTML(html))
    
def make_link(text, link):
    return '<a href="%s" target="_blank">%s</a>' % (link, text)

def show_link(text, link):
    show_html(make_link(text, link))
    
def fullname(user):
    return '%s %s' % (user['first_name'], user['last_name'])

In [ ]:
def get_last_discussions():
    got_everything = False
    results = []
    page = 1

    while not got_everything:
        data = get_data('discussions', page, '-discussion__posted_on')
        data = [(d['id'], d['subject'], d['title'], d['discussion']) for d in data]
        for _id, subject, title, comments in data:
            createds = [dateutil.parser.parse(c['posted_on']) for c in comments]
            created = max(createds)
            got_everything = (created < start_date)
            if not got_everything:
                for comment in comments:
                    if dateutil.parser.parse(comment['posted_on']) >= start_date:
                        results.append((_id, subject, title, comment))
            else:
                break
        if got_everything:
            break
        else:
            page += 1
            print('page', page)

    return results

In [ ]:
def show_users():
    users = get_last_items('users', date_key='since')

    show_html('<h3>%s utilisateurs créés</h3>' % len(users))

    for user in users:
        show_link(fullname(user), user['page'])

In [ ]:
def show_datasets():
    datasets = get_last_items('datasets')

    show_html('<h3>%s jeux de données créés</h3>' % len(datasets))

    for dataset in datasets:
        owner = dataset['organization']['name'] if dataset['organization'] else fullname(dataset['owner'])
        owner_url = dataset['organization']['page'] if dataset['organization'] else dataset['owner']['page']
        html = make_link(dataset['title'], dataset['page'])
        html += ' par '
        html += make_link(owner, owner_url)
        show_html(html)

In [ ]:
def show_orgas():
    orgs = get_last_items('organizations')

    show_html('<h3>%s organisations créées</h3>' % len(orgs))

    for org in orgs:
        show_link(org['name'], org['page'])

In [ ]:
def show_reuses():
    reuses = get_last_items('reuses')

    show_html('<h3>%s réutilisations créées</h3>' % len(reuses))

    for reuse in reuses:
        owner = None
        if reuse['organization']:
            owner = reuse['organization']['name']
        elif reuse['owner']:
            owner = fullname(reuse['owner'])
        owner_url = '#'
        if owner:
            owner_url = reuse['organization']['page'] if reuse['organization'] else reuse['owner']['page']
        html = make_link(reuse['title'], reuse['page'])
        html += ' par '
        html += make_link(owner, owner_url)
        show_html(html)

In [ ]:
def show_discussions():
    discussions = get_last_discussions()

    show_html('<h3>%s commentaires créées</h3>' % len(discussions))

    for d in discussions:
        _id, subject, title, comment = d
        url = '#'
        if subject['class'] == 'Dataset':
            url = 'https://%s/fr/datasets/%s' % (DOMAIN, subject['id'])
        elif subject['class'] == 'Reuse':
            url = 'https://%s/fr/reuses/%s' % (DOMAIN, subject['id'])
        user = make_link(fullname(comment['posted_by']), comment['posted_by']['page'])
        show_html('%s sur %s par %s' % (title, make_link(subject['class'], url), user))
        show_html('<pre>%s</pre>' % comment['content'])
        show_html('<hr/>')